In [20]:
from pathlib import Path

import pandas as pd
import SimpleITK as sitk
import yaml
from imgtools.autopipeline import AutoPipeline

from readii.io import NIFTIWriter
from readii.negative_controls_refactor import NegativeControlManager


In [ ]:
data_dir = Path().cwd().parent / "TRASH" / "data"
INPUT_DATA = data_dir / "dicom"
OUTPUT_DATA = Path('/tmp') / "mit-generated-niftis"
NEGATIVE_CONTROL_OUTPUT_DIR = data_dir / "negative-controls-niftis" 

for d in [data_dir, INPUT_DATA, OUTPUT_DATA]:
    d.mkdir(exist_ok=True)

MODALITIES = "CT,RTSTRUCT"
CT_FILE_NAME = "CT.nii.gz"
ROI_OF_INTEREST = "GTV"
RTSTRUCT_FILE_NAME = f"{ROI_OF_INTEREST}.nii.gz"  # Not used to CREATE, but used to MATCH against med-imagetools

NEGATIVE_CONTROLS = ["sampled", "shuffled", "randomized"]
NEGATIVE_CONTROL_REGIONS = ["roi", "non_roi", "full"]
RANDOM_SEED = 10

roi_matches = {
  ROI_OF_INTEREST: "^(GTV1)$"
}

with Path(INPUT_DATA, "mit_roi_names.yaml").open("w") as outfile:
    yaml.dump(roi_matches, outfile)

pipeline = AutoPipeline(input_directory=INPUT_DATA,
                        output_directory=OUTPUT_DATA,
                        modalities=MODALITIES,
                        spacing=(0., 0., 0.),
                        ignore_missing_regex = True,
                        update=True,
                        read_yaml_label_names = True,
                        roi_yaml_path = Path(INPUT_DATA, "mit_roi_names.yaml")
                        )

pipeline.run()

100%|██████████| 3/3 [00:00<00:00, 4695.12it/s]
/Users/bhklab/dev/radiomics/readii/.pixi/envs/dev/lib/python3.12/site-packages/imgtools/modules/datagraph.py:430: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  relevant_study_id = self.df_new.loc[(self.df_new.edge_type.str.contains(regex_term)), "study_x"].unique()


0_HN-CHUS-0521_HN-CHUS-082

Processing: 1_HN-CHUS-082
Processing: 0_HN-CHUS-052


2024-12-05T17:20:58-0500 [warning  ] Could not get points for ROI `GANGLION2`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-052/StudyUID-94629/RTSTRUCT_SeriesUID-41418/1-1.dcm
2024-12-05T17:20:58-0500 [warning  ] Could not get points for ROI `CTV1`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-052/StudyUID-94629/RTSTRUCT_SeriesUID-41418/1-1.dcm
2024-12-05T17:20:58-0500 [warning  ] Could not get points for ROI `CTV2`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-052/StudyUID-94629/RTSTRUCT_SeriesUID-41418/1-1.dcm
2024-12-05T

0_HN-CHUS-052  start


2024-12-05T17:21:00-0500 [warning  ] Could not get points for ROI `OS CV`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-082/StudyUID-06980/RTSTRUCT_SeriesUID-67882/1-1.dcm
2024-12-05T17:21:00-0500 [warning  ] Could not get points for ROI `OS MAND`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-082/StudyUID-06980/RTSTRUCT_SeriesUID-67882/1-1.dcm
2024-12-05T17:21:00-0500 [warning  ] Could not get points for ROI `OS CRANE LAT`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-082/StudyUID-06980/RTSTRUCT_SeriesUID-67882/1-1.dcm
2024

0_HN-CHUS-052  SAVED IMAGE


2024-12-05T17:21:02-0500 [warning  ] Could not get points for ROI `PACEMAKER`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-082/StudyUID-06980/RTSTRUCT_SeriesUID-67882/1-1.dcm
2024-12-05T17:21:02-0500 [warning  ] Could not get points for ROI `PlanifP1 XTR`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-082/StudyUID-06980/RTSTRUCT_SeriesUID-67882/1-1.dcm
2024-12-05T17:21:02-0500 [warning  ] Could not get points for ROI `PlanifP1 TR2`. [imgtools] call=structureset.from_dicom_rtstruct:48 error=AttributeError("'Dataset' object has no attribute 'ContourSequence'") rtstruct_path=/Users/bhklab/dev/radiomics/readii/TRASH/data/dicom/HN-CHUS-082/StudyUID-06980/RTSTRUCT_SeriesUID-67882/1-1

0_HN-CHUS-052 SAVED MASK ON CT
1_HN-CHUS-082  start
1_HN-CHUS-082  SAVED IMAGE
1_HN-CHUS-082 SAVED MASK ON CT


In [23]:
images_metadata = pd.read_csv(
  OUTPUT_DATA / "dataset.csv",
  index_col=0,
)

###############################################################
# Two writers, one for the original images and one for the negative controls
original_nifti_writer = NIFTIWriter(
  root_directory=NEGATIVE_CONTROL_OUTPUT_DIR,
  filename_format="{SubjectID}/{Modality}/original.nii.gz",
)

neg_nifti_writer = NIFTIWriter(
  root_directory=NEGATIVE_CONTROL_OUTPUT_DIR,
  filename_format="{SubjectID}/{Modality}/{NegativeControl}-{Region}.nii.gz",
)

###############################################################
# Create a NegativeControlManager object
ncm = NegativeControlManager.from_strings(
  negative_control_types=NEGATIVE_CONTROLS,
  region_types=NEGATIVE_CONTROL_REGIONS,
  random_seed=RANDOM_SEED,
)

###############################################################
# iterate over the rows of the dataframe
for row in images_metadata.itertuples():
  base_image = sitk.ReadImage(OUTPUT_DATA / row.output_folder_CT / CT_FILE_NAME)
  mask_image = sitk.ReadImage(OUTPUT_DATA / row.output_folder_RTSTRUCT_CT / RTSTRUCT_FILE_NAME)

  # write the original images again
  original_nifti_writer.save(
    SubjectID=row.Index,
    image=base_image,
    Modality="CT",
  )

  original_nifti_writer.save(
    SubjectID=row.Index,
    image=mask_image,
    Modality="RTSTRUCT",
  )

  # Negative control manager's apply method returns a 
  # tuple of (image: sitk.Image, negative_control: str, region: str)
  for image, negative_control, region in ncm.apply(base_image, mask_image):
    output_nifti_path = neg_nifti_writer.save(
      SubjectID=row.Index,
      image=image,
      NegativeControl=negative_control,
      Region=region,
      Modality="CT",
    )

In [25]:
!tree -F -C -I "*.dcm" $NEGATIVE_CONTROL_OUTPUT_DIR.parent

/Users/bhklab/dev/radiomics/readii/TRASH/data/
├── dicom/
│   ├── HN-CHUS-052/
│   │   └── StudyUID-94629/
│   │       ├── CT_SeriesUID-06362/
│   │       ├── PT_SeriesUID-44600/
│   │       ├── RTDOSE_SeriesUID-11376/
│   │       ├── RTPLAN_SeriesUID-04314/
│   │       ├── RTSTRUCT_SeriesUID-41418/
│   │       └── RTSTRUCT_SeriesUID-87625/
│   ├── HN-CHUS-082/
│   │   └── StudyUID-06980/
│   │       ├── CT_SeriesUID-05195/
│   │       ├── PT_SeriesUID-72508/
│   │       ├── RTDOSE_SeriesUID-89632/
│   │       ├── RTPLAN_SeriesUID-37374/
│   │       ├── RTSTRUCT_SeriesUID-67882/
│   │       └── RTSTRUCT_SeriesUID-91674/
│   └── mit_roi_names.yaml
└── negative-controls-niftis/
    ├── 0_HN-CHUS-052/
    │   ├── CT/
    │   │   ├── original.nii.gz
    │   │   ├── randomized-full.nii.gz
    │   │   ├── randomized-non_roi.nii.gz
    │   │   ├── randomized-roi.nii.gz
    │   │   ├── sampled-full.nii.gz
    │   │   ├── sampled-non_roi.nii.gz
    │   │   ├── sampled-roi.nii.gz
    │   │   ├──

In [29]:
from readii.io.readers import NIFTIReader

original_nifti_reader = NIFTIReader(
			# root_directory=Path("TRASH/data/nifti"),
			root_directory=NEGATIVE_CONTROL_OUTPUT_DIR,
			filename_pattern="{SubjectID}/{Modality}/original.nii.gz",
)

neg_nifti_reader = NIFTIReader(
			# root_directory=Path("TRASH/data/negative-controls-niftis"),
			root_directory=NEGATIVE_CONTROL_OUTPUT_DIR,
			filename_pattern="{SubjectID}/{Modality}/{NegativeControl}-{Region}.nii.gz",
	)

original = original_nifti_reader.map_files()
results = neg_nifti_reader.map_files()

print(original)
print(results)


2024-12-05T17:54:28-0500 [error    ] Filename '1_HN-CHUS-082/CT/shuffled-non_roi.nii.gz' does not match the expected pattern: %(SubjectID)s/%(Modality)s/original.nii.gz [readii] call=readers.extract_metadata:74
2024-12-05T17:54:28-0500 [warning  ] Skipping file /Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/shuffled-non_roi.nii.gz, as it does not match the pattern. [readii] call=readers.map_files:95
2024-12-05T17:54:28-0500 [error    ] Filename '1_HN-CHUS-082/CT/randomized-full.nii.gz' does not match the expected pattern: %(SubjectID)s/%(Modality)s/original.nii.gz [readii] call=readers.extract_metadata:74
2024-12-05T17:54:28-0500 [warning  ] Skipping file /Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/randomized-full.nii.gz, as it does not match the pattern. [readii] call=readers.map_files:95
2024-12-05T17:54:28-0500 [error    ] Filename '1_HN-CHUS-082/CT/randomized-non_roi.nii.gz' does not match the expe

defaultdict(None, {
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/original.nii
.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/RTSTRUCT/origin
al.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'RTSTRUCT'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/0_HN-CHUS-052/CT/original.nii
.gz'): {
        'SubjectID': '0_HN-CHUS-052',
        'Modality': 'CT'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/0_HN-CHUS-052/RTSTRUCT/origin
al.nii.gz'): {
        'SubjectID': '0_HN-CHUS-052',
        'Modality': 'RTSTRUCT'
    }
})

defaultdict(None, {
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/shuffled-non
_roi.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'shuffled',
        'Region': 'non_roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/randomized-f
ull.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'randomized',
        'Region': 'full'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/randomized-n
on_roi.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'randomized',
        'Region': 'non_roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/sampled-full
.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'sampled',
        'Region': 'full'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/randomized-r
oi.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'randomized',
        'Region': 'roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/sampled-non_
roi.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'sampled',
        'Region': 'non_roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/shuffled-ful
l.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'shuffled',
        'Region': 'full'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/sampled-roi.
nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'sampled',
        'Region': 'roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/1_HN-CHUS-082/CT/shuffled-roi
.nii.gz'): {
        'SubjectID': '1_HN-CHUS-082',
        'Modality': 'CT',
        'NegativeControl': 'shuffled',
        'Region': 'roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/0_HN-CHUS-052/CT/shuffled-non
_roi.nii.gz'): {
        'SubjectID': '0_HN-CHUS-052',
        'Modality': 'CT',
        'NegativeControl': 'shuffled',
        'Region': 'non_roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/0_HN-CHUS-052/CT/randomized-f
ull.nii.gz'): {
        'SubjectID': '0_HN-CHUS-052',
        'Modality': 'CT',
        'NegativeControl': 'randomized',
        'Region': 'full'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/0_HN-CHUS-052/CT/randomized-n
on_roi.nii.gz'): {
        'SubjectID': '0_HN-CHUS-052',
        'Modality': 'CT',
        'NegativeControl': 'randomized',
        'Region': 'non_roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/0_HN-CHUS-052/CT/sampled-full
.nii.gz'): {
        'SubjectID': '0_HN-CHUS-052',
        'Modality': 'CT',
        'NegativeControl': 'sampled',
        'Region': 'full'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/0_HN-CHUS-052/CT/randomized-r
oi.nii.gz'): {
        'SubjectID': '0_HN-CHUS-052',
        'Modality': 'CT',
        'NegativeControl': 'randomized',
        'Region': 'roi'
    },
    PosixPath('/Users/bhklab/dev/radiomics/readii/TRASH/data/negative-controls-niftis/0_HN-CHUS-052/CT/sampled-non_
roi.nii.gz'): {
        'SubjectID': '0_HN-CHUS-052',
        'Modality': 'CT',
        'NegativeControl': 'sampled'